# Thông tin sinh viên
* MSSV: 20522012
* Họ và tên: Phạm Trần Anh Tiên
* Bài tập 4
* Ngày thực hiện: 22/3/2022

# Kết nối đến Google Drive và cài đặt một số thư viện cần thiết

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import cv2
import os
import numpy as np
from skimage import feature
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Các hàm tính đặc trưng ảnh

In [ ]:
def TinhHist(pathfilename):
    img = cv2.imread(pathfilename,0)
    hist = cv2.calcHist([img],[0],None,[256],[0,256])
    size = img.shape[0]*img.shape[1]
    hist = hist / size
    return hist

def TinhHog(pathfilename):
    img = cv2.imread(pathfilename,0)
    img = cv2.resize(img, (128, 256))
    (hog, hog_image) = feature.hog(img, orientations=9,
                                                        pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                                                        block_norm='L2-Hys', visualize=True, transform_sqrt=True)
    return hog

def ReadData(path_to_file, option):
    features = []
    labels = []
    for file_name in sorted(os.listdir(path_to_file)):
        if file_name[:3] == 'cat':
            labels.append(0)
        else:
            labels.append(1)
        

        feature = ExtractFeature(os.path.join(path_to_file, file_name), option)
        features.append(feature)

    return features, labels


def ExtractFeature(pathfilename, option):
    if option==0:
        return TinhHist(pathfilename)
    else:
        return TinhHog(pathfilename)

# Đọc dữ liệu

In [ ]:
DATA_DIR = '/content/drive/MyDrive/CS231.M22.KHCL/Baitap/Dataset/dogscats_small'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

In [ ]:
#Histogram
train_hist_features, train_hist_labels = ReadData(path_to_file=TRAIN_DIR,
                                                                option=0)
test_hist_features, test_hist_labels = ReadData(path_to_file=TEST_DIR,
                                                            option=0)

train_hist_features = np.array(train_hist_features)
train_hist_features = np.squeeze(train_hist_features)
train_hist_labels = np.array(train_hist_labels)

test_hist_features = np.array(test_hist_features)
test_hist_features = np.squeeze(test_hist_features)
test_hist_labels = np.array(test_hist_labels)

In [ ]:
#Hog
train_hog_features, train_hog_labels = ReadData(path_to_file=TRAIN_DIR,
                                                                option=1)
test_hog_features, test_hog_labels = ReadData(path_to_file=TEST_DIR,
                                                            option=1)
train_hog_features = np.array(train_hog_features)
train_hog_features = np.squeeze(train_hog_features)
train_hog_labels = np.array(train_hog_labels)

test_hog_features = np.array(test_hog_features)
test_hog_features = np.squeeze(test_hog_features)
test_hog_labels = np.array(test_hog_labels)

# Áp dụng Linear Classification cho đặc trưng Histogram

In [ ]:
clf_model = LogisticRegression(random_state=0).fit(train_hist_features, 
                                                                                      train_hist_labels)

clf_model.predict(train_hist_features[:2, :])
clf_model.predict_proba(test_hist_features[:2, :])
print("Accuracy: ", clf_model.score(test_hist_features, test_hist_labels))

Accuracy:  0.45


# Áp dụng KNN và Linear Classification cho đặc trưng HOG

## KNN

In [ ]:
for i in range(1,6):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(train_hog_features, train_hog_labels)


    acc = model.score(test_hog_features, test_hog_labels)
    print(f'{i} neighbors => accuracy {acc}')

1 neighbors => accuracy 0.45
2 neighbors => accuracy 0.6
3 neighbors => accuracy 0.6
4 neighbors => accuracy 0.55
5 neighbors => accuracy 0.6


## Linear Classification-hog



In [ ]:
clf_model = LogisticRegression(random_state=0).fit(train_hog_features, 
                                                                                      train_hog_labels)

clf_model.predict(train_hog_features[:2, :])
clf_model.predict_proba(test_hog_features[:2, :])
print("Accuracy: ", clf_model.score(test_hog_features, test_hog_labels))

Accuracy:  0.55


# Linear Classification-Histogram